In [4]:
import pandas as pd
import numpy as np
from scipy import stats

# Read the CSV file
df = pd.read_csv('results_sampling_exp/results_single_prompt.csv')

# Define the sim_score columns
sim_score_columns = ['sim_score_1_to', 'sim_score_2_to', 'sim_score_1_and', 
                     'sim_score_2_and', 'sim_score_1_degree', 'sim_score_2_degree']



# Display the first few rows of the dataframe
print("\nFirst few rows of the dataframe:")
df.head()


First few rows of the dataframe:


,trial_num,country_pair,model_name,temperature,sim_score_1_to,sim_score_2_to,sim_score_1_and,sim_score_2_and,sim_score_1_degree,sim_score_2_degree,sim_diff_to,sim_diff_and,sim_diff_degree
0,1,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,10,12,10.0,10.0,0.0,-2,0.0
1,2,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,11,15,10.0,12.0,0.0,-4,-2.0
2,3,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,10,15,10.0,12.0,0.0,-5,-2.0
3,4,U.S.A.-Mexico,gpt-4o,0.8,12.0,12.0,10,10,7.0,9.0,0.0,0,-2.0
4,5,U.S.A.-Mexico,gpt-4o,0.8,8.0,12.0,12,8,10.0,12.0,-4.0,4,-2.0


In [11]:
# Check for outliers and invalid values
for column in sim_score_columns:
    # Check for values outside the range [0, 20]
    out_of_range = df[(df[column] < 0) | (df[column] > 20)]
    if not out_of_range.empty:
        print(f"Outliers in {column} (outside [0, 20] range):")
        print(out_of_range)
    
    # Check for non-whole numbers
    non_whole = df[df[column] != df[column].round()]
    non_whole = non_whole[non_whole[column] % 1 != 0]  # Exclude numbers like 10.0
    if not non_whole.empty:
        print(f"Non-whole numbers in {column}:")
        print(non_whole)

Non-whole numbers in sim_score_1_to:
       trial_num    country_pair   model_name  temperature  sim_score_1_to  \
9762          62   U.S.A.-Mexico  llama3.2_3B          1.2             NaN   
10095         10   U.S.A.-Canada  llama3.2_3B          1.2             NaN   
11091          5  U.S.A.-Germany  llama3.2_3B          1.2             NaN   
11131         45  U.S.A.-Germany  llama3.2_3B          1.2             NaN   
11185         22  U.S.S.R.-Syria  llama3.2_3B          1.2             NaN   
11320          3   U.S.A.-Mexico  llama3.2_3B          1.5             NaN   

       sim_score_2_to  sim_score_1_and  sim_score_2_and  sim_score_1_degree  \
9762             18.0                8               10                17.0   
10095             8.0               15               10                18.0   
11091            14.0               10               15                15.0   
11131            10.0               15                8                14.0   
11185             0.0

In [12]:
print(len(  out_of_range))
print(len(non_whole))
out_of_range.head(20)
# Calculate mean and std of sim_diff columns

4
18


,trial_num,country_pair,model_name,temperature,sim_score_1_to,sim_score_2_to,sim_score_1_and,sim_score_2_and,sim_score_1_degree,sim_score_2_degree,sim_diff_to,sim_diff_and,sim_diff_degree
6986,58,U.S.S.R.-Israel,llama3.2_3B,0.8,0.0,0.0,2,5,2.0,1922.0,0.0,-3,-1920.0
8117,34,U.S.A.-Mexico,llama3.2_3B,1.0,16.0,18.0,18,18,14.0,64.0,-2.0,0,-50.0
9111,27,India-Sri Lanka,llama3.2_3B,1.0,18.0,15.0,15,7,15.0,22315500.0,3.0,8,-22315485.0
11325,8,U.S.A.-Mexico,llama3.2_3B,1.5,14.0,15.0,8,8,12.0,127575529.0,-1.0,0,-127575517.0


In [13]:
non_whole.head(20)

,trial_num,country_pair,model_name,temperature,sim_score_1_to,sim_score_2_to,sim_score_1_and,sim_score_2_and,sim_score_1_degree,sim_score_2_degree,sim_diff_to,sim_diff_and,sim_diff_degree
6488,22,U.S.A.-Mexico,llama3.2_3B,0.8,18.0,18.0,17,10,18.0,NaN,0.0,7,NaN
7698,77,U.S.S.R.-Cuba,llama3.2_3B,0.8,6.0,4.0,8,5,10.0,NaN,2.0,3,NaN
8107,24,U.S.A.-Mexico,llama3.2_3B,1.0,8.0,16.0,10,4,14.0,NaN,-8.0,6,NaN
8152,69,U.S.A.-Mexico,llama3.2_3B,1.0,14.0,12.0,17,9,15.0,NaN,2.0,8,NaN
9714,14,U.S.A.-Mexico,llama3.2_3B,1.2,17.0,17.0,10,10,16.0,NaN,0.0,0,NaN
9764,64,U.S.A.-Mexico,llama3.2_3B,1.2,17.0,12.0,15,9,18.0,NaN,5.0,6,NaN
10186,24,U.S.S.R.-Israel,llama3.2_3B,1.2,5.0,10.0,8,3,2.0,NaN,-5.0,5,NaN
10490,20,Belgium-Luxembourg,llama3.2_3B,1.2,16.0,14.0,5,8,12.0,NaN,2.0,-3,NaN
10705,4,India-Sri Lanka,llama3.2_3B,1.2,15.0,5.0,18,10,16.0,NaN,10.0,8,NaN
10726,25,India-Sri Lanka,llama3.2_3B,1.2,10.0,10.0,11,6,13.0,NaN,0.0,5,NaN


In [14]:
# Filter out non-whole and out of range rows
df_filtered = df.copy()

for column in sim_score_columns:
    # Remove rows with values outside the range [0, 20]
    df_filtered = df_filtered[(df_filtered[column] >= 0) & (df_filtered[column] <= 20)]
    
    # Remove rows with non-whole numbers
    df_filtered = df_filtered[df_filtered[column] == df_filtered[column].round()]

# Print the number of rows removed
print(f"Number of rows in original dataframe: {len(df)}")
print(f"Number of rows in filtered dataframe: {len(df_filtered)}")
print(f"Number of rows removed: {len(df) - len(df_filtered)}")

# Update the dataframe
df = df_filtered

# Display the first few rows of the filtered dataframe
print("\nFirst few rows of the filtered dataframe:")
df.head()


Number of rows in original dataframe: 25006
Number of rows in filtered dataframe: 24876
Number of rows removed: 130

First few rows of the filtered dataframe:


,trial_num,country_pair,model_name,temperature,sim_score_1_to,sim_score_2_to,sim_score_1_and,sim_score_2_and,sim_score_1_degree,sim_score_2_degree,sim_diff_to,sim_diff_and,sim_diff_degree
0,1,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,10,12,10.0,10.0,0.0,-2,0.0
1,2,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,11,15,10.0,12.0,0.0,-4,-2.0
2,3,U.S.A.-Mexico,gpt-4o,0.8,10.0,10.0,10,15,10.0,12.0,0.0,-5,-2.0
3,4,U.S.A.-Mexico,gpt-4o,0.8,12.0,12.0,10,10,7.0,9.0,0.0,0,-2.0
4,5,U.S.A.-Mexico,gpt-4o,0.8,8.0,12.0,12,8,10.0,12.0,-4.0,4,-2.0


In [15]:


# Calculate mean and std of sim_diff columns
sim_diff_columns = ['sim_diff_to', 'sim_diff_and', 'sim_diff_degree']
sim_diff_mean = df[sim_diff_columns].mean()
sim_diff_std = df[sim_diff_columns].std()

print("Mean of sim_diff columns:")
print(sim_diff_mean)
print("\nStandard deviation of sim_diff columns:")
print(sim_diff_std)

# Calculate std of sim_score columns
sim_score_columns = ['sim_score_1_to', 'sim_score_2_to', 'sim_score_1_and', 
                     'sim_score_2_and', 'sim_score_1_degree', 'sim_score_2_degree']
sim_score_std = df[sim_score_columns].std()

print("\nStandard deviation of sim_score columns:")
print(sim_score_std)

# Perform t-test on sim_diff columns
alpha = 0.05
for column in sim_diff_columns:
    t_statistic, p_value = stats.ttest_1samp(df[column], 0)
    print(f"\nt-test results for {column}:")
    print(f"t-statistic: {t_statistic}")
    print(f"p-value: {p_value}")
    if p_value < alpha:
        print("Reject null hypothesis: The difference is statistically significant.")
    else:
        print("Fail to reject null hypothesis: The difference is not statistically significant.")


Mean of sim_diff columns:
sim_diff_to       -0.088961
sim_diff_and      -0.369111
sim_diff_degree   -0.650185
dtype: float64

Standard deviation of sim_diff columns:
sim_diff_to        4.558392
sim_diff_and       4.926690
sim_diff_degree    3.937175
dtype: float64

Standard deviation of sim_score columns:
sim_score_1_to        5.470683
sim_score_2_to        5.384844
sim_score_1_and       5.492062
sim_score_2_and       5.575216
sim_score_1_degree    5.380551
sim_score_2_degree    5.497656
dtype: float64

t-test results for sim_diff_to:
t-statistic: -3.0780767670474867
p-value: 0.0020856785238212113
Reject null hypothesis: The difference is statistically significant.

t-test results for sim_diff_and:
t-statistic: -11.816579416444636
p-value: 3.9064312685904984e-32
Reject null hypothesis: The difference is statistically significant.

t-test results for sim_diff_degree:
t-statistic: -26.04608781962479
p-value: 1.4189838517566537e-147
Reject null hypothesis: The difference is statistically 